In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import xarray as xr
import scipy.stats

### Load and sort data

In [2]:
### Identify sample files
ckl_in = Path('/projects/wakedynamics/orybchuk/ldm-3d/logs/2022-12-01T22-01-37_inpaint_geo_raaw_kl1/images/test/postprocessed')
ckl_files = list(Path(ckl_in).glob('*.npy'))
ckl_files.sort()

In [3]:
### Prepare four large sample arrays
# ncbatch = min(len(ccont_files), len(ucontp_files), len(ukl_files), len(uklp_files))
ncbatch = len(ckl_files)
tmp_samp = np.load(ckl_files[0])
ccond_shape = [ncbatch]+list(tmp_samp.shape)
    
tmp_kl = np.zeros(ccond_shape)
for i in range(ncbatch):
    tmp_kl[i,:,:,:,:] = np.load(ckl_files[i]).copy()

In [4]:
### Convert to Dataset
## Prepare metadata
xdim = np.arange(0, 1920, 15)
ydim = np.arange(0, 1920, 15)
zdim = np.arange(0, 480, 15)  # Chop vertical dim in half to avoid looking above capping inversion
coords = dict(x=xdim, y=ydim, z=zdim)

def unnorm(xp, xmin, xmax):
    '''
    Take data from [-1,1] back to the original values
    '''
    return (xp + 1)*0.5*(xmax-xmin)+xmin

umin, umax, vmin, vmax, wmin, wmax = 2.86975098, 12.5567627, -0.9810791, 4.91235352, -1.98095703, 2.5579834

## Create Dataset
ds_c = xr.Dataset(coords=coords)

ds_c['u_kl'] = (('sample', 'x', 'y', 'z'), unnorm(tmp_kl[:,0,:,:,:32], umin, umax))
ds_c['v_kl'] = (('sample', 'x', 'y', 'z'), unnorm(tmp_kl[:,1,:,:,:32], vmin, vmax))
ds_c['w_kl'] = (('sample', 'x', 'y', 'z'), unnorm(tmp_kl[:,2,:,:,:32], wmin, wmax))

### Velocity profiles

In [5]:
### Calculate stats
for vel in ['u', 'v', 'w']:
    ds_c[vel+'_kl'].mean(('sample', 'x', 'y')).to_netcdf(vel+'_ccond_kl_raaw_mean.nc')
    ds_c[vel+'_kl'].std(('sample', 'x', 'y')).to_netcdf(vel+'_ccond_kl_raaw_std.nc')

### Flux profiles

In [ ]:
### Calculate profiles of kinematic flux
## Calculate fluctuations
ds_c["up_kl"] = ds_c['u_kl'] - ds_c['u_kl'].mean(('x', 'y'))
ds_c["vp_kl"] = ds_c['v_kl'] - ds_c['v_kl'].mean(('x', 'y'))
ds_c["wp_kl"] = ds_c['w_kl'] - ds_c['w_kl'].mean(('x', 'y'))

## Calculate fluxes
ds_c["upvp_kl"] = (ds_c['up_kl'] * ds_c['vp_kl']).mean(('x', 'y'))

ds_c["upwp_kl"] = (ds_c['up_kl'] * ds_c['wp_kl']).mean(('x', 'y'))

ds_c["vpwp_kl"] = (ds_c['vp_kl'] * ds_c['wp_kl']).mean(('x', 'y'))

# Save
ds_c["upvp_kl"].mean('sample').to_netcdf("upvp_ccond_kl_raaw.nc")
ds_c["upwp_kl"].mean('sample').to_netcdf("upwp_ccond_kl_raaw.nc")
ds_c["vpwp_kl"].mean('sample').to_netcdf("vpwp_ccond_kl_raaw.nc")

### Velocity PDFs

In [ ]:
### Calculate histograms and PDFs
## Bins
umin = 6
umax = 12
ubins = np.linspace(umin, umax, 500)

vmin = -1.5
vmax = 3.5
vbins = np.linspace(vmin, vmax, 500)

wmin = -0.75
wmax = 0.75
wbins = np.linspace(wmin, wmax, 300)

## Scipy histograms
uhist_np_kl = np.histogram(ds_c['u_kl'].values.flatten(), bins=ubins)
uhist_kl = scipy.stats.rv_histogram(uhist_np_kl)

vhist_np_kl = np.histogram(ds_c['v_kl'].values.flatten(), bins=vbins)
vhist_kl = scipy.stats.rv_histogram(vhist_np_kl)

whist_np_kl = np.histogram(ds_c['w_kl'].values.flatten(), bins=wbins)
whist_kl = scipy.stats.rv_histogram(whist_np_kl)

## Save
np.save('u_ccond_kl_raaw_hist.npy', uhist_kl.pdf(ubins))
np.save('v_ccond_kl_raaw_hist.npy', vhist_kl.pdf(vbins))
np.save('w_ccond_kl_raaw_hist.npy', whist_kl.pdf(wbins))

### Spectra

In [ ]:
### Calculate 1D spectra, sample by sample
khub = 6
tmp1 = ds_c['up_kl'].isel(sample=0, z=khub)
tmp2 = np.fft.rfftn(tmp1.values, axes=(0,))
all_spectra_kl = np.zeros((len(ds_c['sample']), len(tmp2)))    
all_spectra_klp = np.zeros((len(ds_c['sample']), len(tmp2)))    
    
for i in range(len(ds_c['sample'])):
    up_xy_test = ds_c['up_kl'].isel(sample=i, z=khub)
    up_k_test = np.fft.rfftn(up_xy_test.values, axes=(0,))
    up_k_bar_test = np.mean(np.abs(up_k_test)**2, axis=1)
    all_spectra_kl[i,:] = up_k_bar_test.copy()
    
np.save('spectra_ccond_kl_raaw.npy', all_spectra_kl.mean(axis=0))

### Continuity

In [ ]:
### Calculate pdf's of gradients + divergence
## Parameters
dx, dy, dz = 15, 15, 15
facx, facy, facz = 1/(4*dx), 1/(4*dy), 1/(4*dz)
dmin = -0.02
dmax = 0.02
dbins = np.linspace(dmin, dmax, 500)

## Set up history arrays
all_dudx_hist_kl = np.zeros((len(ds_c['sample']), len(dbins)))
all_dvdy_hist_kl = np.zeros((len(ds_c['sample']), len(dbins)))
all_dwdz_hist_kl = np.zeros((len(ds_c['sample']), len(dbins)))
all_div_hist_kl = np.zeros((len(ds_c['sample']), len(dbins)))

## Calculate pdf's
for i in range(len(ds_c['sample'])):
    vel = ds_c.isel(sample=i)
    
    # Calculate gradients + divergence
    dudx_kl = facx*(-vel['u_kl'].roll(x=-1,y=-1,z=-1).values + vel['u_kl'].roll(x=0,y=-1,z=-1).values \
                    -vel['u_kl'].roll(x=-1,y=0 ,z=-1).values + vel['u_kl'].roll(x=0,y=0 ,z=-1).values \
                    -vel['u_kl'].roll(x=-1,y=-1,z=0 ).values + vel['u_kl'].roll(x=0,y=-1,z=0 ).values \
                    -vel['u_kl'].roll(x=-1,y=0 ,z=0 ).values + vel['u_kl'].roll(x=0,y=0 ,z=0 ).values)
    dvdy_kl = facy*(-vel['v_kl'].roll(x=-1,y=-1,z=-1).values - vel['v_kl'].roll(x=0,y=-1,z=-1).values \
                    +vel['v_kl'].roll(x=-1,y=0 ,z=-1).values + vel['v_kl'].roll(x=0,y=0 ,z=-1).values \
                    -vel['v_kl'].roll(x=-1,y=-1,z=0 ).values - vel['v_kl'].roll(x=0,y=-1,z=0 ).values \
                    +vel['v_kl'].roll(x=-1,y=0 ,z=0 ).values + vel['v_kl'].roll(x=0,y=0 ,z=0 ).values)
    dwdz_kl = facz*(-vel['w_kl'].roll(x=-1,y=-1,z=-1).values - vel['w_kl'].roll(x=0,y=-1,z=-1).values \
                    -vel['w_kl'].roll(x=-1,y=0 ,z=-1).values - vel['w_kl'].roll(x=0,y=0 ,z=-1).values \
                    +vel['w_kl'].roll(x=-1,y=-1,z=0 ).values + vel['w_kl'].roll(x=0,y=-1,z=0 ).values \
                    +vel['w_kl'].roll(x=-1,y=0 ,z=0 ).values + vel['w_kl'].roll(x=0,y=0 ,z=0 ).values)
    div_kl = dudx_kl + dvdy_kl + dwdz_kl
    
    # Calculate histograms
    duhist_kl_np = np.histogram(dudx_kl.flatten(), bins=dbins)
    duhist_kl = scipy.stats.rv_histogram(duhist_kl_np)  
    dvhist_kl_np = np.histogram(dvdy_kl.flatten(), bins=dbins)
    dvhist_kl = scipy.stats.rv_histogram(dvhist_kl_np)  
    dwhist_kl_np = np.histogram(dwdz_kl.flatten(), bins=dbins)
    dwhist_kl = scipy.stats.rv_histogram(dwhist_kl_np) 
    ddivhist_kl_np = np.histogram(div_kl.flatten(), bins=dbins)
    ddivhist_kl = scipy.stats.rv_histogram(ddivhist_kl_np) 
    
    # Store
    all_dudx_hist_kl[i,:] = duhist_kl.pdf(dbins)
    all_dvdy_hist_kl[i,:] = dvhist_kl.pdf(dbins)
    all_dwdz_hist_kl[i,:] = dwhist_kl.pdf(dbins)
    all_div_hist_kl[i,:] = ddivhist_kl.pdf(dbins)
    
np.save('dudx_ccond_kl_raaw_hist.npy', all_dudx_hist_kl.mean(axis=0))
np.save('dvdy_ccond_kl_raaw_hist.npy', all_dvdy_hist_kl.mean(axis=0))
np.save('dwdz_ccond_kl_raaw_hist.npy', all_dwdz_hist_kl.mean(axis=0))
np.save('div_ccond_kl_raaw_hist.npy', all_div_hist_kl.mean(axis=0))